In [48]:
import numpy as np
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
import os
import random
from difflib import SequenceMatcher

In [60]:


CONV_FILE = "conversations.json"   # will store list of { "user": "...", "response": "...", "agent": "default"}
AGENTS_FILE = "agents.json"        # will store dict of agents keyed by name


class BulkChatBot:
    def __init__(self, default_agent="default"):
        self.default_agent = default_agent
        self.conversations = []
        self.agents = {}


        self._load_files()
        if self.default_agent not in self.agents:

            self.add_agent(self.default_agent, persona="داعم،الرحمة")

        print(" بوت التحميل الجاهز – المحادثات:", len(self.conversations), "الـ agents:", len(self.agents))

    def _load_files(self):
        if os.path.exists(CONV_FILE):
            try:
                with open(CONV_FILE, "r", encoding="utf-8") as f:
                    self.conversations = json.load(f)
            except Exception:
                self.conversations = []
        if os.path.exists(AGENTS_FILE):
            try:
                with open(AGENTS_FILE, "r", encoding="utf-8") as f:
                    self.agents = json.load(f)
            except Exception:
                self.agents = {}

    def _save_files(self):
        with open(CONV_FILE, "w", encoding="utf-8") as f:
            json.dump(self.conversations, f, ensure_ascii=False, indent=2)
        with open(AGENTS_FILE, "w", encoding="utf-8") as f:
            json.dump(self.agents, f, ensure_ascii=False, indent=2)

    def add_agent(self, name, persona):
        """
        Add a new agent/persona.
        `name`: short id (e.g. "rahma", "therapist_v1")
        `persona`: text describing style (used for metadata)
        """
        self.agents[name] = {"name": name, "persona": persona}
        self._save_files()
        print(f" Agent '{name}' added.")

    def list_agents(self):
        return list(self.agents.keys())

    def add_entries(self, entries, agent=None, save=True):
        """
        Add many conversation entries at once.
        entries: list of tuples or dicts.
          - tuple/list format: (user_text, response_text)
          - dict format: {"user": "...", "response": "...", optional "agent": "..."}
        agent: default agent name to attach if entries don't include agent.
        """
        added = 0
        for e in entries:
            if isinstance(e, (list, tuple)) and len(e) >= 2:
                user_text, response_text = e[0].strip(), e[1].strip()
                ag = agent or self.default_agent
            elif isinstance(e, dict) and "user" in e and "response" in e:
                user_text, response_text = e["user"].strip(), e["response"].strip()
                ag = e.get("agent", agent or self.default_agent)
            else:
                continue
            self.conversations.append({"user": user_text, "response": response_text, "agent": ag})
            added += 1
        if save and added:
            self._save_files()
        print(f" Added {added} entries (agent={agent or self.default_agent}).")

    def load_from_csv(self, path, agent=None, delimiter="||"):
        """
        Load simple CSV/text lines where each line: user{delimiter}response
        e.g. "أنا حزين||ولا تزعل، أنا معك"
        """
        if not os.path.exists(path):
            print("File not found:", path); return
        lines = open(path, "r", encoding="utf-8").read().splitlines()
        entries = []
        for ln in lines:
            if delimiter in ln:
                a, b = ln.split(delimiter, 1)
                entries.append((a.strip(), b.strip()))
        self.add_entries(entries, agent=agent)
        print(" Loaded from", path)


    @staticmethod
    def _similar(a, b):
        return SequenceMatcher(None, a, b).ratio()

    def find_best_response(self, user_text, agent=None, top_k=1, min_score=0.4):
        """
        Find best matching responses among stored conversations.
        Returns best response (string) or None.
        Uses simple fuzzy matching (SequenceMatcher). You can tune min_score.
        """
        agent = agent or self.default_agent
        best = []
        for entry in self.conversations:
            if entry.get("agent") != agent:
                continue
            score = self._similar(user_text, entry["user"])
            best.append((score, entry["response"], entry["user"]))
        if not best:
            return None
        best.sort(reverse=True, key=lambda x: x[0])
        if best[0][0] >= min_score:
            return best[0][1]  # return response
        return None

    def start_console_chat(self, agent=None):
        """
        Start console chat. Agent optional to choose persona.
        """
        agent = agent or self.default_agent
        persona = self.agents.get(agent, {}).get("persona", "")
        print(f"--- بدء المحادثة مع '{agent}' ({persona}) ---")
        print('اكتب "agent:اسم" لتبديل الشخصية، اكتب "exit" أو "خلص" للخروج.')

        while True:
            text = input("أنت >> ").strip()
            if not text:
                continue
            if text.lower() in ["خلص", "exit", "مع السلامة"]:
                print("البوت >> بخاطرك ")
                break

            if text.startswith("agent:"):
                candidate = text.split("agent:",1)[1].strip()
                if candidate in self.agents:
                    agent = candidate
                    print(f" تم التبديل للشخصية '{agent}'")
                else:
                    print("الشخصية غير موجودة. المتاحة:", self.list_agents())
                continue

            resp = self.find_best_response(text, agent=agent, min_score=0.45)
            if resp:
                print("البوت >>", resp)
            else:

                fallback = [
                    "أنا سامعك 🤍، كمل كلامك لو حابب.",
                    "ولا يهمك، إحكيلي أكثر لو بترتاح.",
                    "معك دايمًا، احكيلي شنو مضايقك.",
                    "آه، هالشي صعب.. بس إنت قوي."
                ]
                print("البوت >>", random.choice(fallback))


    def stats(self):
        by_agent = {}
        for c in self.conversations:
            a = c.get("agent", self.default_agent)
            by_agent[a] = by_agent.get(a, 0) + 1
        return {"total_conversations": len(self.conversations), "agents": list(self.agents.keys()), "counts": by_agent}



if __name__ == "__main__":
    bot = BulkChatBot()


    bulk = [
        ("أنا خايف", "طبيعي تحس بالخوف، بدنا نطمن شوي. احكيلي السبب؟"),
        ("أنا تعبان", "ارتاحي شوي وخلي بالك بنفسك، شو سبب التعب؟"),
        ("أنا جائع", "رغم الظروف، حاول تأمون حاجة بسيطة لو تقدر. ربي معك."),

    ]
    bot.add_entries(bulk, agent="default")


    bot.add_agent("rahma_support", "رحمة – صوت حنون، دافئ، يواسي ويعطي خطوات بسيطة")


    bot.start_console_chat()


 بوت التحميل الجاهز – المحادثات: 0 الـ agents: 2
 Added 3 entries (agent=default).
 Agent 'rahma_support' added.
--- بدء المحادثة مع 'default' (داعم، لطيف، يستمع ويعطي تشجيع قصير باللهجة الفلسطينية) ---
اكتب "agent:اسم" لتبديل الشخصية، اكتب "exit" أو "خلص" للخروج.
أنت >> مرحبا
البوت >> ولا يهمك، إحكيلي أكثر لو بترتاح.
أنت >> السلام عليكم 
البوت >> أنا سامعك 🤍، كمل كلامك لو حابب.
أنت >> تحفيز
البوت >> معك دايمًا، احكيلي شنو مضايقك.


KeyboardInterrupt: Interrupted by user